In [115]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
# Set up Chrome with options
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")  # Fake user-agent
options.add_argument("--incognito")  # Open in incognito mode

driver = webdriver.Chrome( options=options)



In [123]:

import time


def scrape_remotive_jobs(title):
    url = f"https://remotive.io/remote-jobs?query={title}"
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    try:
        # Wait for job listings container
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#hits > ul > div[x-data]")))

        # Try selecting "Sort by newest"
        try:
            sort_by_dropdown = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#sort-by select')))
            select = Select(sort_by_dropdown)
            select.select_by_index(1)  # Sort by Newest
            time.sleep(3)  # Wait for sorting to apply
        except Exception:
            print("Sort dropdown not found, skipping sorting step.")

        # Fetch job items **AFTER** sorting
        job_items = driver.find_elements(By.CSS_SELECTOR, '#hits > ul > div[x-data]')

        jobs = []
        for job in job_items:
            try:
                # Extract job link
                job_link = job.find_element(By.CSS_SELECTOR, 'a.remotive-url-visit').get_attribute('href')

                # Extract title and company name
                title_element = job.find_elements(By.CSS_SELECTOR, '.remotive-bold')
                job_title = title_element[0].text if len(title_element) > 0 else "Unknown"
                company_name = title_element[-1].text if len(title_element) > 1 else "Not specified"

                # Extract job location (fallback to "Remote")
                job_tags = job.find_elements(By.CSS_SELECTOR, '.tag-small')
                location = job_tags[0].text if job_tags else "Remote"

                # Extract posting date
                posted = "Not provided"
                posted_elements = job.find_elements(By.XPATH, './/div[contains(@class, "tw-hidden sm:tw-flex")]/span/span')
                if posted_elements:
                    posted = posted_elements[0].text.strip()

                # Extract company logo
                img_element = job.find_elements(By.CSS_SELECTOR, 'img')
                img_src = img_element[0].get_attribute('src') if img_element else "No Image"

                # Store job details as dictionary
                job_as_JSON = {
                    "title": job_title,
                    "company": company_name,
                    "location": location,
                    "posted": posted,
                    "href": job_link,
                    "type": "Remote",
                    "img": img_src
                }
                jobs.append(job_as_JSON)

            except Exception as e:
                print(f"Error extracting job details: {e}")

        # print(jobs)
        return jobs

    except Exception as e:
        print("Job listings not found or error:", e)
        return []

# Call function
scrape_remotive_jobs("Engineer")




[{'title': 'Senior Frontend Developer',
  'company': 'Mimo GmbH',
  'location': 'Software Development',
  'posted': 'Not provided',
  'href': 'https://remotive.com/remote-jobs/software-dev/senior-frontend-developer-1968223',
  'type': 'Remote',
  'img': 'https://remotive.com/web/image/hr.job/1968223/logo/64x64'},
 {'title': 'Senior Backend Developer',
  'company': 'Mimo GmbH',
  'location': 'Software Development',
  'posted': 'Not provided',
  'href': 'https://remotive.com/remote-jobs/software-dev/senior-backend-developer-1968220',
  'type': 'Remote',
  'img': 'https://remotive.com/web/image/hr.job/1968220/logo/64x64'},
 {'title': 'Senior Independent Software Developer',
  'company': 'A.Team',
  'location': 'Software Development',
  'posted': 'Not provided',
  'href': 'https://remotive.com/remote-jobs/software-dev/senior-independent-software-developer-1919265',
  'type': 'Remote',
  'img': 'https://remotive.com/web/image/hr.job/1919265/logo/64x64'},
 {'title': 'Senior Independent UX/UI